[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Jydn9vGCp9zCW3laYFYeUZOnK1h7kR6o)

Author:
- **Safouane El Ghazouali**,
- Ph.D. in AI,
- Senior data scientist and researcher at TOELT LLC,
- Lecturer at HSLU

# -----  -----  -----  -----  -----  -----  -----  -----

# 🛠️ Finetuning YOLOv10 on a Custom Dataset

Welcome to this hands-on notebook on finetuning YOLOv10 using the Ultralytics library. We'll use a custom dataset from Kaggle in YOLO format to adapt the model for specific object detection tasks.

![YOLO Finetuning](https://so-development.org/wp-content/uploads/2025/02/ultralytics-yolo-1024x538.jpg)

### Why Finetune?
- **Domain Adaptation**: Tailor pre-trained models to your specific data.
- **Improved Accuracy**: Enhance performance on custom objects/classes.
- **Efficiency**: Leverage transfer learning to reduce training time and resources.

### What You'll Learn
- Downloading and preparing a Kaggle dataset in YOLO format.
- Finetuning YOLOv10 on the custom dataset.
- Evaluating the finetuned model.
- Exploring the output dictionary of results.

# 🧰 Environment Setup

Install required libraries, including Ultralytics and Kaggle for dataset download.

In [ ]:
!pip install ultralytics kaggle matplotlib opencv-python

### Set Up Kaggle API

To download the dataset, you'll need a Kaggle API key. Follow these steps:
1. Go to Kaggle > Account > Create New API Token to download `kaggle.json`.
2. Upload `kaggle.json` in the next cell.

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print('Kaggle API set up successfully!')

### Import Libraries

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
print("Libraries imported successfully!")

# 📂 Downloading the Custom Dataset

We'll use the 'Object Detection - Wildlife Dataset - YOLO Format' from Kaggle, which contains 1500 images across 4 classes (buffalo, elephant, rhino, zebra) in YOLO format.

In [ ]:
!kaggle datasets download -d ankanghosh651/object-detection-wildlife-dataset-yolo-format
!unzip -q object-detection-wildlife-dataset-yolo-format.zip
print('Dataset downloaded and unzipped!')

# Explanation
# - The dataset includes train/val/test splits with images and YOLO-format labels (.txt files).
# - A data.yaml file should be present or created to define paths and classes.

### Prepare data.yaml

If data.yaml is not included, create it. Adjust paths as needed.

In [ ]:
yaml_content = """
path: ./final_data  # Root path
train: train/images
val: valid/images
test: test/images

nc: 4  # Number of classes
names: ['buffalo', 'elephant', 'rhino', 'zebra']  # Class names
"""

with open('data.yaml', 'w') as f:
    f.write(yaml_content)
print('data.yaml created!')

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt

# Paths
images_dir = "./final_data/train/images"
labels_dir = "./final_data/train/labels"

# Get a sample of images
image_files = os.listdir(images_dir)[:5]  # first 5 images

# Class names (same as in data.yaml)
class_names = ['buffalo', 'elephant', 'rhino', 'zebra']

# Assign one fixed BGR color per class
class_colors = {
    0: (255, 0, 0),   # buffalo → Blue
    1: (0, 255, 0),   # elephant → Green
    2: (0, 0, 255),   # rhino → Red
    3: (255, 255, 0)  # zebra → Cyan
}

def plot_image_with_boxes(image_path, label_path):
    # Read image
    img = cv2.imread(image_path)
    h, w, _ = img.shape

    # Read labels (YOLO format: class x_center y_center width height)
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f.readlines():
                cls, x, y, bw, bh = map(float, line.strip().split())
                cls = int(cls)

                # Convert YOLO format (relative) to pixel coordinates
                x1 = int((x - bw/2) * w)
                y1 = int((y - bh/2) * h)
                x2 = int((x + bw/2) * w)
                y2 = int((y + bh/2) * h)

                # Get class-specific color
                color = class_colors.get(cls, (255, 255, 255))

                # Draw rectangle & class label
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, class_names[cls], (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Convert BGR → RGB for matplotlib
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis("off")
    plt.show()

# Show a few images
for file in image_files:
    img_path = os.path.join(images_dir, file)
    label_path = os.path.join(labels_dir, file.rsplit(".", 1)[0] + ".txt")  # match extensions
    plot_image_with_boxes(img_path, label_path)


# 📦 Loading the Model for Finetuning

Load a pre-trained YOLOv10 nano model.

In [ ]:
model = YOLO('yolov10n.pt')
print('YOLOv10 model loaded!')

# 🏋️ Finetuning the Model

Finetune on the custom dataset with specified parameters.

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

results = model.train(
    data='./data.yaml',
    epochs=10,  # Small for demo; increase for better results
    imgsz=640,
    batch=8,
    name='finetuned_wildlife',
    device=device  # Use 'cuda' if GPU available
)

# Explanation
# - data: Path to YAML defining dataset structure.
# - epochs: Training iterations.
# - Results are saved in 'runs/detect/finetuned_wildlife'.

# 📊 Evaluating the Finetuned Model

Validate on the validation set.

In [ ]:
from PIL import Image

val_results = model.val()
print(val_results)

# Visualize metrics if images are generated
if os.path.exists('runs/detect/finetuned_wildlife/confusion_matrix.png'):
    img = Image.open('runs/detect/finetuned_wildlife/confusion_matrix.png')
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# 🔍 Inference with Finetuned Model

Test on a sample test image and explore the output.

In [ ]:
# Assume a test image path; adjust as needed
test_img_path = 'final_data/test/images/0557.jpg'
results_test = model(test_img_path)

# Visualize
annotated = results_test[0].plot()
plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
plt.title('Finetuned YOLOv10 Detections')
plt.axis('off')
plt.show()

## 📊 Analyzing Output Dictionary

Explore the results structure, including bounding boxes and confidences.

In [ ]:
result = results_test[0]

boxes = result.boxes.xyxy
confidences = result.boxes.conf
classes = result.boxes.cls

print("\nBounding Boxes (xyxy):\n", boxes)
print("Confidences:\n", confidences)
print("Classes:\n", classes)

class_names = result.names
detected = [class_names[int(cls)] for cls in classes]
print("Detected Classes:\n", detected)

# 🧠 Interpreting Results

Review mAP, precision, and recall from validation. Compare bounding boxes and confidences to pre-trained outputs.

# 💡 Student Task (**HOME**)

1. Increase epochs and retrain for better accuracy.
2. Try a different model size (e.g., 'yolov11s.pt').
3. Use another Kaggle dataset and adapt the YAML.
4. Explore additional training args (check out https://docs.ultralytics.com/modes/train/#resuming-interrupted-trainings).